<a href="https://colab.research.google.com/github/AUT-Student/BigData-HW1/blob/main/BigData_HW1_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><b>In the name of God</b></center>

<b>Course</b>: ‌Big Data
<br>
<b>Description:</b> HomeWork 1 | Question 1
<br>
<b>Developer</b>: Alireza Mazochi (400131075)

# PySpark Setting

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 28 kB/s 
     |████████████████████████████████| 198 kB 52.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=b1781dc1aee088381c44ed1e8ab5f7278dca36f64ecb155cbccaf91d802e6b3e
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [3]:
spark

In [6]:
import numpy as np
1 not in np.array([1,2,3])

False

# Load and Prepare Dataset

In [7]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id='1-OCBGBtKoY_PadKHcXDyWxHQ2BS8Nulo'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('hw1.zip')

!unzip hw1.zip

Archive:  hw1.zip
   creating: hw1-files/
   creating: hw1-files/q3/
  inflating: hw1-files/q3/patches.csv  
  inflating: hw1-files/q3/lsh.py     
   creating: hw1-files/q1/
  inflating: hw1-files/q1/dataset1.txt  
   creating: hw1-files/q1/.ipynb_checkpoints/
   creating: hw1-files/q2/
  inflating: hw1-files/q2/games_library.txt  
   creating: hw1-files/.ipynb_checkpoints/


In [8]:
def convert_string_to_list(string):
  if string=="":
    return []
  else:
    return [int(item) for item in string.split(",")]

In [9]:
import pandas as pd
df = pd.read_csv("/content/hw1-files/q1/dataset1.txt", sep="\t",names=["id", "friend_id"])

In [10]:
df = df.fillna("")
df["friend_id"] = df["friend_id"].apply(convert_string_to_list)

In [20]:
df[df["id"]==98]["friend_id"].values

array([list([6, 95, 5937, 7468, 7985, 11841, 20003, 29784, 32198, 35679, 35740, 41765, 43140, 44279, 44505, 49909, 575, 2831, 3781, 5933, 7989, 13421, 19662, 20562, 23173, 23318, 24609, 28381, 44431, 48141])],
      dtype=object)

In [17]:
df[df["id"]==575]

,id,friend_id
4110,575,"[522, 6, 98, 571, 578, 621, 1382, 2495, 2782, ..."


In [11]:
rdd=spark.sparkContext.parallelize(df.to_dict('records'), 2)

# Map-Reduce

In [21]:
def map_function(item):
  fids = sorted(item["friend_id"])
  output = []
    
  for fid1 in fids:
    counter = {}
    for fid2 in fids:

      if fid1 != fid2:
        counter[fid2] = 1
    
    output.append((fid1, counter))

  return output

In [22]:
from heapq import merge

def reduce_function(value1, value2):
  merged_ids = list(merge(value1.keys(), value2.keys()))

  merged_value = {}
  for id in merged_ids:
    merged_value[id] = value1.get(id, 0) + value2.get(id, 0)

  return merged_value

In [23]:
mapped_rdd = rdd.flatMap(map_function)

In [24]:
reduced_rdd = mapped_rdd.reduceByKey(reduce_function)

In [25]:
output_rdd = reduced_rdd.collect()

In [35]:
def suggest_new_friend(id, number_top):
  exist_friend_id = df[df["id"]==id]["friend_id"].values[0]

  for item in output_rdd:
    if item[0]==id:
      counter = item[1]
      break

  sorted_new_friend = sorted(counter, key=counter.get, reverse=True)

  suggestion_list = []

  for new_id in sorted_new_friend:
    if new_id not in exist_friend_id:
      suggestion_list.append(new_id)
      if len(suggestion_list) == number_top:
        break

  return suggestion_list

# Example

In [36]:
for id in [98, 135, 117, 911, 8804]:
  print(f"id = {id}")
  print(f"Suggestion = {suggest_new_friend(id,10)}\n")

id = 98
Suggestion = [18560, 2554, 13654, 16324, 16350, 30134, 30691, 17, 113, 134]

id = 135
Suggestion = [13792, 33060, 629, 5490, 19217, 25256, 34151, 34164, 34441, 45054]

id = 117
Suggestion = [34164, 12519, 13793, 15314, 23507, 23510, 25256, 34140, 34169, 34207]

id = 911
Suggestion = [24456, 39540, 40560, 30984, 30993, 30995, 30996, 33333, 37875, 41352]

id = 8804
Suggestion = [34179, 34332, 3230, 8677, 11399, 11400, 13182, 13872, 15207, 29745]

